In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import plotly.express as px

2024-03-18 19:31:52.775536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'tensorflow.keras.layers.experimental'

In [ ]:
def prepare_data():
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train = x_train.reshape((x_train.shape[0], -1))
    x_test = x_test.reshape((x_test.shape[0], -1))
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
def display_images(x_data):
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    for i in range(5):
        axes[i].imshow(x_data[i].reshape(28, 28), cmap='gray')
        axes[i].axis('off')
    plt.show()

In [ ]:
def display_class_distribution(y_data):
    class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
    class_counts = np.sum(y_data, axis=0)

    fig = px.pie(names=class_names, values=class_counts, title='Répartition des classes dans le dataset')
    fig.show()

In [ ]:
def create_model(num_hidden_layers, num_neurons):
    model = Sequential()
    model.add(Rescaling(1.0 / 255, input_shape=(784,)))
    for _ in range(num_hidden_layers):
        model.add(Dense(num_neurons, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def train_model(model, x_train, y_train, x_val, y_val, epochs=30, batch_size=32):
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

    # Courbe d'apprentissage (Loss)
    plt.plot(history.history['loss'], label='Training loss')
    plt.plot(history.history['val_loss'], label='Validation loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    # Courbe d'apprentissage (Accuracy)
    plt.plot(history.history['accuracy'], label='Training accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation accuracy')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

    return history


In [ ]:
def evaluate_model(model, x_test, y_test):
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")
    print(f"Test Loss: {test_loss:.4f}")

    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    print("Matrice de Confusion:")
    print(conf_matrix)

    class_report = classification_report(y_test_classes, y_pred_classes)
    print("Rapport de Classification:")
    print(class_report)

In [ ]:
def main(configurations):
    for config in configurations:
        num_hidden_layers, num_neurons, epochs = config

        x_train, y_train, x_val, y_val, x_test, y_test = prepare_data()
        display_images(x_train.reshape(-1, 28, 28))
        display_class_distribution(y_train)

        model = create_model(num_hidden_layers, num_neurons)
        history = train_model(model, x_train, y_train, x_val, y_val, epochs=epochs)
        evaluate_model(model, x_test, y_test)

In [ ]:
if __name__ == "__main__":
    # Exemple de configurations. Vous pouvez ajuster ces valeurs selon vos besoins.
    configurations = [
        (2, 64, 30),
        (3, 128, 50),
        # Ajoutez d'autres configurations au besoin
    ]

    main(configurations)